In [8]:
import mod_sql as m_s
import pandas as pd
import imp
imp.reload(m_s)

<module 'mod_sql' from '/Users/chulwonchae/UBION/220919/mod_sql/__init__.py'>

In [9]:
## 클래스 생성
db = m_s.Database()

In [10]:
## 단순 행 결합
## python에서는 concat()
## sql union 결합
sql = """
        select * from transaction_1 
        union 
        select * from transaction_2
"""
db.executeall(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


### 굉장히 단순하지만 노가다성이 뛰어난 sql
1. transaction_1, transaction_2를 union 결합
2. transaction_detail_1, transaction_detail_2를 union 결합
3. 1번 2번 과정에서 나온 테이블을 left join
4. 이 모든 과정이 sql문 한번에 처리 


hint (  select * from 1번 테이블 left join 2번 테이블 on 조건  )

In [11]:
# 2번 과정 
sql = """
        select * from transaction_detail_1 
        union 
        select * from transaction_detail_2
"""
db.executeall(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [12]:
sql = """
    select 
    * 
    from 
    (select * from transaction_1 
    union 
    select * from transaction_2) as tr 
    left join 
    (select * from transaction_detail_1 
    union
    select * from transaction_detail_2) as tr_d 
    on 
    tr.transaction_id = tr_d.transaction_id
"""
db.executeall(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tr_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1


### pandas에서 작업
1. 4개의 테이블을 각각 변수에 담아서 시작
2. concat()과 merge() 함수를 이용하여 sql의 결과와 같은 출력

In [13]:
sql = "select * from transaction_1"
sql2 = "select * from transaction_2"
sql3 = "select * from transaction_detail_1"
sql4 = "select * from transaction_detail_2"

tr1 = db.executeall(sql)
tr2 = db.executeall(sql2)
tr_d1 = db.executeall(sql3)
tr_d2 = db.executeall(sql4)

In [14]:
tr = pd.concat([tr1, tr2], ignore_index=True)
tr_d = pd.concat([tr_d1, tr_d2], ignore_index=True)

In [15]:
df = pd.merge(tr, tr_d, on='transaction_id', how='left')
df

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


In [16]:
tr1 = pd.read_csv("../csv/transaction_1.csv")
tr2 = pd.read_csv("../csv/transaction_2.csv")

In [17]:
pd.concat([tr1, tr2], ignore_index=True)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [18]:
## left join transaction_detail_1, item_master 결합
## quantity * item_price 결과값을 price 컬럼에 삽입
## 결과적으로 컬럼 출력
## transaction_id, 
## quantity, 
## item_price, 
## price
## 4개의 컬럼을 출력
sql = "select * from transaction_detail_1"
sql2 = "select * from item_master"

In [19]:
sql = """
    select 
    transaction_id, 
    quantity, 
    item_price, 
    tr_d.item_id, 
    (quantity * item_price) as price 
    from 
    transaction_detail_1 as tr_d 
    left join 
    item_master as im 
    on 
    tr_d.item_id = im.item_id
"""
db.executeall(sql)

,transaction_id,quantity,item_price,item_id,price
0,T0000000113,1,210000,S005,210000
1,T0000000114,1,50000,S001,50000
2,T0000000115,1,120000,S003,120000
3,T0000000116,1,210000,S005,210000
4,T0000000117,2,85000,S002,170000
...,...,...,...,...,...
4995,T0000004865,1,120000,S003,120000
4996,T0000004866,3,50000,S001,150000
4997,T0000004867,3,50000,S001,150000
4998,T0000004868,1,210000,S005,210000


In [20]:
sql = """
        select 
        payment_date, 
        DATE_FORMAT(payment_date, '%%Y%%m') as payment_month 
        from 
        transaction_1
"""
db.executeall(sql)

,payment_date,payment_month
0,2019-02-01 01:36:57,201902
1,2019-02-01 01:37:23,201902
2,2019-02-01 02:34:19,201902
3,2019-02-01 02:47:23,201902
4,2019-02-01 04:33:46,201902
...,...,...
4995,2019-06-15 02:42:41,201906
4996,2019-06-15 03:36:16,201906
4997,2019-06-15 03:44:06,201906
4998,2019-06-15 04:14:06,201906
